# Deploying NVIDIA Triton Inference Server in AI Platform Prediction Custom Container (Google Cloud SDK)

In this notebook, we will walk through the process of deploying NVIDIA's Triton Inference Server into AI Platform Prediction Custom Container service in the Direct Model Server mode:

![](img/caip_triton_container_diagram_direct.jpg)


In [ ]:
PROJECT_ID='[Enter project name - REQUIRED]'
REPOSITORY='caipcustom'
REGION='us-central1'
TRITON_VERSION='20.06'

In [ ]:
import os
import random
import requests
import json

MODEL_BUCKET='gs://{}-{}'.format(PROJECT_ID,random.randint(10000,99999))
ENDPOINT='https://{}-ml.googleapis.com/v1'.format(REGION)
TRITON_IMAGE='tritonserver:{}-py3'.format(TRITON_VERSION)
CAIP_IMAGE='{}-docker.pkg.dev/{}/{}/{}'.format(REGION,PROJECT_ID,REPOSITORY,TRITON_IMAGE)

In [1]:
'''
# Test values

PROJECT_ID='tsaikevin-1236'
REPOSITORY='caipcustom'
REGION='us-central1'
TRITON_VERSION='20.06'

import os
import random
import requests
import json

MODEL_BUCKET='gs://{}-{}'.format(PROJECT_ID,random.randint(10000,99999))
ENDPOINT='https://{}-ml.googleapis.com/v1'.format(REGION)
TRITON_IMAGE='tritonserver:{}-py3'.format(TRITON_VERSION)
CAIP_IMAGE='{}-docker.pkg.dev/{}/{}/{}'.format(REGION,PROJECT_ID,REPOSITORY,TRITON_IMAGE)
'''

In [2]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


### Create the Artifact Registry
This will be used to store the container image for the model server Triton.

In [3]:
!gcloud beta artifacts repositories create $REPOSITORY --repository-format=docker --location=$REGION

ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [4]:
!gcloud beta auth configure-docker $REGION-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


### Prepare the container
We will make a copy of the Triton container image into the Artifact Registry, where AI Platform Custom Container Prediction will only pull from during Model Version setup. The following steps will download the NVIDIA Triton Inference Server container to your VM, then upload it to your repo.

In [5]:
!docker pull nvcr.io/nvidia/$TRITON_IMAGE && \
 docker tag nvcr.io/nvidia/$TRITON_IMAGE $CAIP_IMAGE && \
 docker push $CAIP_IMAGE

20.06-py3: Pulling from nvidia/tritonserver
Digest: sha256:36f94c39221c4e19921d44296690991057bbebbb15f59dacd88e25ff331bd307
Status: Image is up to date for nvcr.io/nvidia/tritonserver:20.06-py3
nvcr.io/nvidia/tritonserver:20.06-py3
The push refers to repository [us-central1-docker.pkg.dev/tsaikevin-1236/caipcustom/tritonserver]

7aefd4ea: Preparing 
ab22f50a: Preparing 
4bb8a14c: Preparing 
c357696a: Preparing 
35b111ce: Preparing 
422b8a56: Preparing 
5c73ed66: Preparing 
91761c8c: Preparing 
dcbd0b8f: Preparing 
3fad0b37: Preparing 
bca7086a: Preparing 
a1fe0dac: Preparing 
16262158: Preparing 
faf9c798: Preparing 
4dd7a77b: Preparing 
4f618f62: Preparing 
114ab5c3: Preparing 
b7588393: Preparing 
7a4b3a0b: Preparing 
3708beeb: Preparing 
c2e3c7b1: Preparing 
43d8d50a: Preparing 
9bd9798f: Preparing 
27c9414b: Preparing 
4c1700eb: Preparing 
46c23e3a: Preparing 
b877a610: Preparing 
e28a7437: Preparing 
d0584a68: Preparing 
cd722629: Preparing 
7d6d8dba: Preparing 
f0c094ce: Preparin

### Prepare model Artifacts
Clone the NVIDIA Triton Inference Server repo.

In [6]:
!git clone -b r$TRITON_VERSION https://github.com/triton-inference-server/server.git

Cloning into 'server'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 25522 (delta 27), reused 37 (delta 15), pack-reused 25460
Receiving objects: 100% (25522/25522), 14.27 MiB | 22.48 MiB/s, done.
Resolving deltas: 100% (18876/18876), done.


Create the GCS bucket where the model artifacts will be copied to.

In [7]:
!gsutil mb $MODEL_BUCKET

Creating gs://tsaikevin-1236-21636/...


Stage model artifacts and copy to bucket.

In [8]:
!mkdir model_repository

In [ ]:
'''
%cd triton-inference-server
!git checkout r$TRITON_VERSION
%cd ..
'''

In [9]:
!cp -R server/docs/examples/model_repository/* model_repository/

In [10]:
!./server/docs/examples/fetch_models.sh

+ mkdir -p model_repository/resnet50_netdef/1
+ wget -O model_repository/resnet50_netdef/1/model.netdef http://download.caffe2.ai.s3.amazonaws.com/models/resnet50/predict_net.pb
--2020-10-27 04:40:42--  http://download.caffe2.ai.s3.amazonaws.com/models/resnet50/predict_net.pb
Resolving download.caffe2.ai.s3.amazonaws.com (download.caffe2.ai.s3.amazonaws.com)... 52.216.244.196
Connecting to download.caffe2.ai.s3.amazonaws.com (download.caffe2.ai.s3.amazonaws.com)|52.216.244.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31649 (31K) [binary/octet-stream]
Saving to: ‘model_repository/resnet50_netdef/1/model.netdef’

model_repository/re 100%[===================>]  30.91K  --.-KB/s    in 0.03s   

2020-10-27 04:40:43 (1.11 MB/s) - ‘model_repository/resnet50_netdef/1/model.netdef’ saved [31649/31649]

+ wget -O model_repository/resnet50_netdef/1/init_model.netdef http://download.caffe2.ai.s3.amazonaws.com/models/resnet50/init_net.pb
--2020-10-27 04:40:43--  http

In [11]:
!gsutil -m cp -R model_repository/ $MODEL_BUCKET

Copying file://model_repository/simple/config.pbtxt [Content-Type=application/octet-stream]...
Copying file://model_repository/simple/1/model.graphdef [Content-Type=application/octet-stream]...
Copying file://model_repository/inception_graphdef/inception_labels.txt [Content-Type=text/plain]...
Copying file://model_repository/resnet50_netdef/config.pbtxt [Content-Type=application/octet-stream]...
Copying file://model_repository/inception_graphdef/config.pbtxt [Content-Type=application/octet-stream]...
Copying file://model_repository/inception_graphdef/1/model.graphdef [Content-Type=application/octet-stream]...
Copying file://model_repository/resnet50_netdef/1/model.netdef [Content-Type=application/octet-stream]...
Copying file://model_repository/resnet50_netdef/resnet50_labels.txt [Content-Type=text/plain]...
Copying file://model_repository/resnet50_netdef/1/init_model.netdef [Content-Type=application/octet-stream]...
Copying file://model_repository/densenet_onnx/config.pbtxt [Content-T

In [12]:
!gsutil ls -RLl $MODEL_BUCKET/model_repository

gs://tsaikevin-1236-21636/model_repository/:

gs://tsaikevin-1236-21636/model_repository/densenet_onnx/:
       387  2020-10-27T04:40:51Z  gs://tsaikevin-1236-21636/model_repository/densenet_onnx/config.pbtxt
     10311  2020-10-27T04:40:51Z  gs://tsaikevin-1236-21636/model_repository/densenet_onnx/densenet_labels.txt

gs://tsaikevin-1236-21636/model_repository/densenet_onnx/1/:
  32719461  2020-10-27T04:40:52Z  gs://tsaikevin-1236-21636/model_repository/densenet_onnx/1/model.onnx

gs://tsaikevin-1236-21636/model_repository/inception_graphdef/:
       340  2020-10-27T04:40:51Z  gs://tsaikevin-1236-21636/model_repository/inception_graphdef/config.pbtxt
     10329  2020-10-27T04:40:51Z  gs://tsaikevin-1236-21636/model_repository/inception_graphdef/inception_labels.txt

gs://tsaikevin-1236-21636/model_repository/inception_graphdef/1/:
  95746802  2020-10-27T04:40:52Z  gs://tsaikevin-1236-21636/model_repository/inception_graphdef/1/model.graphdef

gs://tsaikevin-1236-21636/model_repository

### Prepare request payload

To prepare the payload format, we have included a utility get_request_body_simple.py.  To use this utility, install the following library:

In [13]:
!pip3 install geventhttpclient

#### Prepare non-binary request payload

The first model will illustrate a non-binary payload.  The following command will create a KF Serving v2 format non-binary payload to be used with the "simple" model:

In [14]:
!python3 get_request_body_simple.py -m simple

#### Prepare binary request payload

Triton's implementation of KF Serving v2 protocol for binary data appends the binary data after the json body.  Triton requires an additional header for offset:

`Inference-Header-Content-Length: [offset]`

We have provided a script that will automatically resize the image to the proper size for ResNet-50 [224, 224, 3] and calculate the proper offset.  The following command takes an image file and outputs the necessary data structure to be use with the "resnet50_netdef" model.  Please note down this offset as it will be used later.

In [15]:
!python3 get_request_body_simple.py -m image -f server/qa/images/mug.jpg

(3, 224, 224)
Add Header: Inference-Header-Content-Length: 138


## Create and deploy Model and Model Version

In this section, we will deploy two models:
1. Simple model with non-binary data.  KF Serving v2 protocol specifies a json format with non-binary data in the json body itself.
2. Binary data model with ResNet-50.  Triton's implementation of binary data for KF Server v2 protocol.


### Simple model (non-binary data)

#### Create Model

AI Platform Prediction uses a Model/Model Version Hierarchy, where the Model is a logical grouping of Model Versions.  We will first create the Model.

Because the MODEL_NAME variable will be used later to specify the predict route, and Triton will use that route to run prediction on a specific model, we must set the value of this variable to a valid name of a model.  For this section, will use the "simple" model.

In [16]:
MODEL_NAME='simple'

In [17]:
!gcloud ai-platform models create $MODEL_NAME --region $REGION --enable-logging

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ml engine model [projects/tsaikevin-1236/models/simple].


In [18]:
!gcloud ai-platform models list --region $REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
NAME    DEFAULT_VERSION_NAME
simple


#### Create Model Version

After the Model is created, we can now create a Model Version under this Model.  Each Model Version will need a name that is unique within the Model.  In AI Platform Prediction Custom Container, a {Project}/{Model}/{ModelVersion} uniquely identifies the specific container and model artifact used for inference.

In [19]:
VERSION_NAME='v01'

The following config file will be used in the Model Version creation command.

#### Command with YAML config file

In [20]:
import yaml

config_simple={'deploymentUri': MODEL_BUCKET+'/model_repository', \
               'container': {'image': CAIP_IMAGE, \
                             'args': ['tritonserver', '--model-repository=$(AIP_STORAGE_URI)'], \
                             'env': [], \
                             'ports': {'containerPort': 8000}}, \
               'routes': {'predict': '/v2/models/'+MODEL_NAME+'/infer', \
                          'health': '/v2/models/'+MODEL_NAME}, \
               'machineType': 'n1-standard-4', 'autoScaling': {'minNodes': 1}}

with open(r'config_simple.yaml', 'w') as file:
    config = yaml.dump(config_simple, file)

In [21]:
!gcloud beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --accelerator count=1,type=nvidia-tesla-t4 \
  --config config_simple.yaml \
  --region=$REGION \
  --async

Using endpoint [https://us-central1-ml.googleapis.com/]


#### To see details of the Model Version just created

In [32]:
!gcloud ai-platform versions describe $VERSION_NAME --model=$MODEL_NAME --region=$REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
acceleratorConfig:
  count: '1'
  type: NVIDIA_TESLA_T4
autoScaling:
  minNodes: 1
container:
  args:
  - tritonserver
  - --model-repository=$(AIP_STORAGE_URI)
  image: us-central1-docker.pkg.dev/tsaikevin-1236/caipcustom/tritonserver:20.06-py3
  ports:
  - containerPort: 8000
createTime: '2020-10-27T04:41:21Z'
deploymentUri: gs://tsaikevin-1236-21636/model_repository
etag: 56sj7hcwcL0=
isDefault: true
machineType: n1-standard-4
name: projects/tsaikevin-1236/models/simple/versions/v01
routes:
  health: /v2/models/simple
  predict: /v2/models/simple/infer
state: READY


#### To list all Model Versions and their states in this Model

In [33]:
!gcloud ai-platform versions list --model=$MODEL_NAME --region=$REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
NAME  DEPLOYMENT_URI                              STATE
v01   gs://tsaikevin-1236-21636/model_repository  READY


#### Run prediction using `curl`

The "simple" model takes two tensors with shape [1,16] and does a couple of basic arithmetic operation.

In [34]:
!curl -X POST $ENDPOINT/projects/$PROJECT_ID/models/$MODEL_NAME/versions/$VERSION_NAME:predict \
    -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    -d @simple.json

{"id":"0","model_name":"simple","model_version":"1","outputs":[{"name":"OUTPUT0","datatype":"INT32","shape":[1,16],"data":[-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]},{"name":"OUTPUT1","datatype":"INT32","shape":[1,16],"data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]}]}

In [35]:
# this doesn't: gcloud auth application-default print-access-token

!curl -X POST $ENDPOINT/projects/$PROJECT_ID/models/$MODEL_NAME/versions/$VERSION_NAME:predict \
    -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth application-default print-access-token`" \
    -d @simple.json

{
  "error": {
    "code": 403,
    "message": "Access to model denied.",
    "status": "PERMISSION_DENIED"
  }
}


#### Run prediction using Using `requests` library

In [36]:
with open('simple.json', 'r') as s:
    data=s.read()
    
PREDICT_URL = "{}/projects/{}/models/{}/versions/{}:predict".format(ENDPOINT, PROJECT_ID, MODEL_NAME, VERSION_NAME)
HEADERS = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer {}'.format(os.popen('gcloud auth print-access-token').read().rstrip())
}

response = requests.request("POST", PREDICT_URL, headers=HEADERS, data = data).content.decode()

json.loads(response)

{'id': '0',
 'model_name': 'simple',
 'model_version': '1',
 'outputs': [{'name': 'OUTPUT0',
   'datatype': 'INT32',
   'shape': [1, 16],
   'data': [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]},
  {'name': 'OUTPUT1',
   'datatype': 'INT32',
   'shape': [1, 16],
   'data': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]}]}

In [37]:
# this doesn't work: gcloud auth application-default print-access-token

with open('simple.json', 'r') as s:
    data=s.read()
    
PREDICT_URL = "https://us-central1-ml.googleapis.com/v1/projects/tsaikevin-1236/models/simple/versions/v01:predict"
HEADERS = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer {}'.format(os.popen('gcloud auth application-default print-access-token').read().rstrip())
}

response = requests.request("POST", PREDICT_URL, headers=HEADERS, data = data).content.decode()

json.loads(response)

{'error': {'code': 403,
  'message': 'Access to model denied.',
  'status': 'PERMISSION_DENIED'}}

### ResNet-50 model (binary data)

#### Create Model

In [25]:
BINARY_MODEL_NAME='resnet50_netdef'

In [26]:
!gcloud ai-platform models create $BINARY_MODEL_NAME --region $REGION --enable-logging

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ml engine model [projects/tsaikevin-1236/models/resnet50_netdef].


#### Create Model Version

In [27]:
BINARY_VERSION_NAME='v01'

#### Command with YAML config file

In [28]:
import yaml

config_binary={'deploymentUri': MODEL_BUCKET+'/model_repository', \
               'container': {'image': CAIP_IMAGE, \
                             'args': ['tritonserver', '--model-repository=$(AIP_STORAGE_URI)'], \
                             'env': [], \
                             'ports': {'containerPort': 8000}}, \
               'routes': {'predict': '/v2/models/'+BINARY_MODEL_NAME+'/infer', \
                          'health': '/v2/models/'+BINARY_MODEL_NAME}, \
               'machineType': 'n1-standard-4', 'autoScaling': {'minNodes': 1}}

with open(r'config_binary.yaml', 'w') as file:
    config_binary = yaml.dump(config_binary, file)

In [29]:
!gcloud beta ai-platform versions create $BINARY_VERSION_NAME \
  --model $BINARY_MODEL_NAME \
  --accelerator count=1,type=nvidia-tesla-t4 \
  --config config_binary.yaml \
  --region=$REGION \
  --async

Using endpoint [https://us-central1-ml.googleapis.com/]


#### To see details of the Model Version just created

In [38]:
!gcloud ai-platform versions describe $BINARY_VERSION_NAME --model=$BINARY_MODEL_NAME --region=$REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
acceleratorConfig:
  count: '1'
  type: NVIDIA_TESLA_T4
autoScaling:
  minNodes: 1
container:
  args:
  - tritonserver
  - --model-repository=$(AIP_STORAGE_URI)
  image: us-central1-docker.pkg.dev/tsaikevin-1236/caipcustom/tritonserver:20.06-py3
  ports:
  - containerPort: 8000
createTime: '2020-10-27T04:42:01Z'
deploymentUri: gs://tsaikevin-1236-21636/model_repository
etag: RTRC-Os6Aa0=
isDefault: true
machineType: n1-standard-4
name: projects/tsaikevin-1236/models/resnet50_netdef/versions/v01
routes:
  health: /v2/models/resnet50_netdef
  predict: /v2/models/resnet50_netdef/infer
state: READY


#### To list all Model Versions and their states in this Model

In [39]:
!gcloud ai-platform versions list --model=$BINARY_MODEL_NAME --region=$REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
NAME  DEPLOYMENT_URI                              STATE
v01   gs://tsaikevin-1236-21636/model_repository  READY


#### Run prediction using `curl`

Recall the offset value calcuated above.  The binary case has an additional header:

`Inference-Header-Content-Length: [offset]`

In [40]:
!curl --request POST $ENDPOINT/projects/$PROJECT_ID/models/$BINARY_MODEL_NAME/versions/$BINARY_VERSION_NAME:predict \
    -k -H "Content-Type: application/octet-stream" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    -H "Inference-Header-Content-Length: 138" \
    --data-binary @payload.dat

{"id":"0","model_name":"resnet50_netdef","model_version":"1","outputs":[{"name":"gpu_0/softmax","datatype":"FP32","shape":[1,1000],"data":[9.698576386441005e-10,3.2031421959288765e-9,1.9225324110472287e-10,1.622270651147062e-9,4.734711511744649e-10,4.736601777466376e-9,5.2833907387972448e-11,1.5530178254508087e-9,1.0723617549501797e-9,1.6877773845180856e-8,2.7639702082637997e-8,2.4067116299875125e-8,2.3789946013152986e-8,3.813896753968038e-8,1.707913277471107e-8,1.677788752374454e-8,2.7277375913570269e-9,7.533176749063842e-9,3.448606022971035e-8,1.5721646207111918e-8,3.0091542591037525e-9,1.8809401258756964e-10,2.4593422853769199e-9,2.8420139575047189e-9,1.3236193252552653e-9,9.352436158849287e-9,1.2529297599428446e-8,8.686888541831195e-9,8.706960485937998e-9,5.088249110940524e-8,1.4554123461962832e-9,6.056438461854441e-9,7.962206893807889e-9,7.382499944696974e-9,3.922888236473909e-9,1.672104676941899e-9,6.8971197642042629e-9,3.3895479756296256e-9,1.5144490106422382e-9,1.02825759018543

In [41]:
# this doesn't work: gcloud auth application-default print-access-token

!curl --request POST $ENDPOINT/projects/$PROJECT_ID/models/$BINARY_MODEL_NAME/versions/$BINARY_VERSION_NAME:predict \
    -k -H "Content-Type: application/octet-stream" \
    -H "Authorization: Bearer `gcloud auth application-default print-access-token`" \
    -H "Inference-Header-Content-Length: 138" \
    --data-binary @payload.dat

{
  "error": {
    "code": 403,
    "message": "Access to model denied.",
    "status": "PERMISSION_DENIED"
  }
}


#### Run prediction using Using `requests` library

In [44]:
with open('payload.dat', 'rb') as s:
    data=s.read()
    
PREDICT_URL = "{}/projects/{}/models/{}/versions/{}:predict".format(ENDPOINT, PROJECT_ID, BINARY_MODEL_NAME, BINARY_VERSION_NAME)
HEADERS = {
  'Content-Type': 'application/octet-stream',
  'Inference-Header-Content-Length': '138',
  'Authorization': 'Bearer {}'.format(os.popen('gcloud auth print-access-token').read().rstrip())
}

response = requests.request("POST", PREDICT_URL, headers=HEADERS, data = data).content.decode()

json.loads(response)

{'id': '0',
 'model_name': 'resnet50_netdef',
 'model_version': '1',
 'outputs': [{'name': 'gpu_0/softmax',
   'datatype': 'FP32',
   'shape': [1, 1000],
   'data': [9.698576386441005e-10,
    3.2031421959288764e-09,
    1.9225324110472286e-10,
    1.622270651147062e-09,
    4.734711511744649e-10,
    4.736601777466376e-09,
    5.2833907387972445e-11,
    1.5530178254508087e-09,
    1.0723617549501796e-09,
    1.6877773845180855e-08,
    2.7639702082637996e-08,
    2.4067116299875124e-08,
    2.3789946013152985e-08,
    3.813896753968038e-08,
    1.707913277471107e-08,
    1.677788752374454e-08,
    2.7277375913570268e-09,
    7.533176749063841e-09,
    3.448606022971035e-08,
    1.5721646207111917e-08,
    3.0091542591037523e-09,
    1.8809401258756964e-10,
    2.4593422853769198e-09,
    2.8420139575047187e-09,
    1.3236193252552653e-09,
    9.352436158849287e-09,
    1.2529297599428446e-08,
    8.686888541831195e-09,
    8.706960485937998e-09,
    5.088249110940524e-08,
    1.45541

In [43]:
# this doesn't work: gcloud auth application-default print-access-token

with open('payload.dat', 'rb') as s:
    data=s.read()
    
PREDICT_URL = "{}/projects/{}/models/{}/versions/{}:predict".format(ENDPOINT, PROJECT_ID, BINARY_MODEL_NAME, BINARY_VERSION_NAME)
HEADERS = {
  'Content-Type': 'application/octet-stream',
  'Inference-Header-Content-Length': '138',
  'Authorization': 'Bearer {}'.format(os.popen('gcloud auth application-default print-access-token').read().rstrip())
}

response = requests.request("POST", PREDICT_URL, headers=HEADERS, data = data).content.decode()

json.loads(response)

{'error': {'code': 403,
  'message': 'Access to model denied.',
  'status': 'PERMISSION_DENIED'}}

## Clean up

In [ ]:
!gcloud ai-platform versions delete $VERSION_NAME --model=$MODEL_NAME --region=$REGION --quiet

In [ ]:
!gcloud ai-platform models delete $MODEL_NAME --region=$REGION --quiet

In [ ]:
!gcloud ai-platform versions delete $BINARY_VERSION_NAME --model=$BINARY_MODEL_NAME --region=$REGION --quiet

In [ ]:
!gcloud ai-platform models delete $BINARY_MODEL_NAME --region=$REGION --quiet

In [ ]:
!gsutil -m rm -r -f $MODEL_BUCKET

In [ ]:
!rm -rf model_repository triton-inference-server server *.yaml *.dat *.json